# Tensorflow ImageNet-ResNet50 Patch Demo

This notebook demonstrates the adversarial patch attack applied on the ResNet50 model, and **it is assumed that you have a copy of the ImageNet dataset and have access to a CUDA-compatible GPU (this demo cannot be run on a typical personal computer)**.
Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.


## Setup: Experiment Name and ImageNet Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

In [ ]:
# Import packages from the Python standard library
import importlib.util
import sys
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "imagenet_adversarial_patches"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/dioptra/data/ImageNet-Kaggle"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_RESTAPI_URI") is None:
    os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [ ]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [ ]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [ ]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])

    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)

    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])

    return job_response


# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)

    if(job_response['status']=="failed"):
        return {}

    run = mlflow_client.get_run(job_response["mlflowRunId"])

    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])

    return run


def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## Adversarial Patches: ImageNet Evaluation

Now, we will test the pretrained ResNet50 model on a subset of the ImageNet dataset.
We will be submitting our job to the `"tensorflow_gpu"` queue.

In [ ]:
response_resnet50_init = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        f"-P register_model_name={EXPERIMENT_NAME}_pretrained_resnet50",
        "-P batch_size=20",
        "-P imagenet_preprocessing=True",
        "-P image_size=224,224,3",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Initialization job for ResNet50 neural network submitted")
print("")
pprint.pprint(response_resnet50_init)

response_resnet50_init = get_run_id(response_resnet50_init)

### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [ ]:
# Create Patches
response_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_init["jobId"],
)

print("Patch attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_patch)
print("")

response_resnet50_patch = get_run_id(response_resnet50_patch)

In [ ]:
# Deploy Patch attack on test set.
response_deploy_resnet50_patch_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_resnet50_patch['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=500",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_patch["jobId"],
)

print("Patch deployment (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_deploy_resnet50_patch_testing)
print("")

response_deploy_resnet50_patch_testing = get_run_id(response_deploy_resnet50_patch_testing)

In [ ]:
# Check patched dataset results
response_resnet50_infer_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_resnet50_patch_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            "-P batch_size=100",
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_resnet50_patch_testing["jobId"],
)

print("Patch evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_infer_resnet50_patch)
print("")

response_resnet50_infer_resnet50_patch = get_run_id(response_resnet50_infer_resnet50_patch)
print_mlflow_results(response_resnet50_infer_resnet50_patch)